In [1]:
import pandas as pd
import numpy as np
import os
import time
import cv2
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import random

In [2]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense
from keras.layers import Flatten, Reshape, Activation, Concatenate, Dropout

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [69]:
FOLDER = '../../../data_handson/kuzushiji/'
IMAGES = FOLDER + 'train_images/'
CHARS = FOLDER + 'chars/'
print(os.listdir(FOLDER))

['.DS_Store', 'chars', 'unicode_translation.csv', 'image', 'train.csv', 'train_images', 'sample_submission.csv']


In [4]:
# os.mkdir(CHARS`)
# for code in unicodes:
#     os.mkdir(CHARS + code + '/' )

In [5]:
df_train = pd.read_csv(FOLDER + 'train.csv')
df_sub = pd.read_csv(FOLDER + 'sample_submission.csv')
unicode_map = {codepoint: char for codepoint, char in pd.read_csv(FOLDER + 'unicode_translation.csv').values}

In [7]:
df_sub_idx = df_sub.set_index("image_id")
df_train_idx = df_train.set_index("image_id")
idx_train = df_train['image_id'][:500]

In [8]:
def label_reader(label):
    try:
        code_arr = np.array(label['labels'].split(' ')).reshape(-1, 5)
    except:
        return
    return code_arr

In [10]:
df_char_train = pd.DataFrame()
start = time.time()
for idx in idx_train[:100]:
    code = df_train_idx.loc[idx]
    try:
        code_arr = np.array(code['labels'].split(' ')).reshape(-1, 5)
    except:
        continue
    df_char = pd.DataFrame(code_arr, columns=['unicode', 'x', 'y', 'w', 'h'])
    df_char['image_id'] = idx
    df_char_train = pd.concat([df_char_train, df_char], axis=0)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:0.2274928092956543[sec]


In [11]:
idx = idx_train[0]
code = df_train_idx.loc[idx]
code_arr = np.array(code['labels'].split(' ')).reshape(-1, 5)
df_char = pd.DataFrame(code_arr, columns=['unicode', 'x', 'y', 'w', 'h'])
df_char[['x','y','w','h']] = df_char[['x','y','w','h']].astype('int')
df_char.head()

,unicode,x,y,w,h
0,U+306F,1231,3465,133,53
1,U+304C,275,1652,84,69
2,U+3044,1495,1218,143,69
3,U+3051,220,3331,53,91
4,U+306B,911,1452,61,92


In [12]:
CHAR_SIZE = (100, 100)
for idx in tqdm(idx_train[:1000]):
    code = df_train_idx.loc[idx]
    try:
        code_arr = np.array(code['labels'].split(' ')).reshape(-1, 5)
    except:
        continue
    df_char = pd.DataFrame(code_arr, columns=['unicode', 'x', 'y', 'w', 'h'])
    df_char[['x','y','w','h']] = df_char[['x','y','w','h']].astype('int')

    path = IMAGES + idx + '.jpg'
    img = cv2.imread(path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    codes = df_char['unicode']
    positions = df_char[['x','y', 'w','h']].values
    char_arr = np.zeros([0, CHAR_SIZE[0], CHAR_SIZE[1]])
    for pos in positions :
        x, y, w, h = pos
        img_char = img_gray[y:y+h, x:x+w]
        if w > h:
            delta = (w - h) // 2
            pad = (np.ones([delta, w]) * 255).astype('int')
            img_pad = (255 - np.vstack([pad, img_char, pad])).astype('float32')        
            img_pad_resize = cv2.resize(img_pad, CHAR_SIZE).astype('int')
            char_arr = np.vstack([char_arr, img_pad_resize.reshape([1, CHAR_SIZE[0], CHAR_SIZE[1]])])
        else:
            delta = (h - w) // 2
            pad = (np.ones([h, delta]) * 255).astype('int')
            img_pad = (255 - np.hstack([pad, img_char, pad])).astype('float32')
            img_pad_resize = cv2.resize(img_pad, CHAR_SIZE).astype('int')        
            char_arr = np.vstack([char_arr, img_pad_resize.reshape([1, CHAR_SIZE[0], CHAR_SIZE[1]])])

    with open(CHARS + idx + '.pickle', 'wb') as f:
        pickle.dump(char_arr, f)
        pickle.dump(codes, f)

100%|██████████| 500/500 [03:33<00:00,  2.34it/s]


In [13]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense
from keras.layers import Flatten, Reshape, Activation, Concatenate, Dropout

In [85]:
img_size = (CHAR_SIZE[0], CHAR_SIZE[1], 1)
inputs = Input(shape=img_size, name='input')

## Block 1
conv1_1 = Conv2D(32, (3, 3),activation='relu',padding='same',name='conv1_1')(inputs)
conv1_2 = Conv2D(32, (3, 3),activation='relu',padding='same',name='conv1_2')(conv1_1)
pool1 = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool1')(conv1_2)

conv2_1 = Conv2D(32, (3, 3),activation='relu',padding='same',name='conv2_1')(pool1)
conv2_2 = Conv2D(32, (3, 3),activation='relu',padding='same',name='conv2_2')(conv2_1)
pool2 = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool2')(conv2_2)

conv3_1 = Conv2D(32, (3, 3),activation='relu',padding='same',name='conv3_1')(pool2)
conv3_2 = Conv2D(32, (3, 3),activation='relu',padding='same',name='conv3_2')(conv3_1)
pool3 = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool3')(conv3_2)

flat = Flatten(name='flatten')(pool3)
dense1 = Dense(4781, name='dense1')(flat)
dense2 = Dense(4781, name='dense2')(dense1)
pred = Activation('softmax',name='pred')(dense2)
model =  Model(inputs, pred)

In [86]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 100, 100, 1)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 100, 100, 32)      320       
_________________________________________________________________
conv1_2 (Conv2D)             (None, 100, 100, 32)      9248      
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 50, 50, 32)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 50, 50, 32)        9248      
_________________________________________________________________
conv2_2 (Conv2D)             (None, 50, 50, 32)        9248      
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 25, 25, 32)        0         
__________

In [87]:
from keras.losses import categorical_crossentropy
from keras.optimizers import adam
from sklearn.preprocessing import  OneHotEncoder

In [88]:
model.compile(optimizer=adam(),
              loss=categorical_crossentropy)

ohe = OneHotEncoder()
ohe.fit(pd.Series(list(unicode_map.keys())).unique().reshape(-1,1))

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [89]:
pd.Series(list(unicode_map.keys())).unique().reshape(-1,1).shape

(4781, 1)

In [117]:
def data_flow(CHARS, idx, ohe, batch_size=32):
    while True:
        image_id = random.sample(list(idx),1)[0]
        
        try:
            with open(CHARS + image_id + '.pickle', 'rb') as f:
                char_arr = pickle.load(f) / 255
                codes = pickle.load(f)
        except:
            continue
            
        num = len(codes)
        try:
            pickup = random.sample(list(range(num)), batch_size)
        except:
            continue
            
        inputs = char_arr[pickup][:, :, :, np.newaxis]
        targets = ohe.transform(codes[pickup].values[:, np.newaxis]).toarray()
        
        yield inputs, targets

In [118]:
len_train = len(idx_train)
idx_rand = random.sample(list(idx_train), len_train)
idx_dev = idx_rand[:int(len_train*0.8)]
idx_val = idx_rand[int(len_train*0.8):]

In [119]:
gen_train = data_flow(CHARS, idx_dev, ohe, batch_size=32)
gen_val = data_flow(CHARS, idx_val, ohe, batch_size=32)

In [120]:
num_steps_dev = len(idx_dev)
num_steps_val = len(idx_val)

In [ ]:
model.fit_generator(gen_train, steps_per_epoch=num_steps_dev, epochs=1, verbose=1, 
                    validation_data=gen_val, 
                    validation_steps=num_steps_val)

Epoch 1/1
 36/400 [=>............................] - ETA: 7:47 - loss: 6.3079